In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import numpy as np
import requests
import time

# Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text

# Prepare the dataset
sequence_length = 20
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}
encoded_text = [char_to_int[ch] for ch in text]

# Create sequences and targets
sequences = []
targets = []
for i in range(0, len(encoded_text) - sequence_length):
    seq = encoded_text[i:i+sequence_length]
    target = encoded_text[i+sequence_length]
    sequences.append(seq)
    targets.append(target)

# Convert to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

# Define the dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# Split the dataset into training and testing sets
dataset = CharDataset(sequences, targets)
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [7]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, nhead):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output[:, -1, :])  # Get the output of the last Transformer block
        return output


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_size = 128
num_layers = 3
num_head = 2
learning_rate = 0.001
epochs = 10
model = TransformerModel(len(chars), hidden_size, len(chars), num_layers, num_head)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
import time
start = time.time()
for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Move data to device
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)

        epoch_train_loss = train_loss / len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)  # Move data to device
                val_output = model(inputs)
                loss = criterion(val_output, targets)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(val_output, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        epoch_val_loss = val_loss / len(test_loader.dataset)
        epoch_val_accuracy = correct / total

        if (epoch+1) % 1 == 0:
            print(f'Epoch {epoch+1}, Train Loss: {epoch_train_loss}, Validation Loss: {epoch_val_loss}, Validation Accuracy: {epoch_val_accuracy}')
end =time.time()
print("Total time :", end-start)

Epoch 1, Train Loss: 2.519549455334185, Validation Loss: 2.4880177918279918, Validation Accuracy: 0.2681295528409728
Epoch 2, Train Loss: 2.4874208505338884, Validation Loss: 2.4760990618772336, Validation Accuracy: 0.268676454107363
Epoch 3, Train Loss: 2.4816236661176196, Validation Loss: 2.474202023235442, Validation Accuracy: 0.27184130897680153
Epoch 4, Train Loss: 2.4793150275861997, Validation Loss: 2.470321856289985, Validation Accuracy: 0.2705009525944189
Epoch 5, Train Loss: 2.4763068766100793, Validation Loss: 2.47033997193909, Validation Accuracy: 0.269765773842878
Epoch 6, Train Loss: 2.4786783396005965, Validation Loss: 2.468403602842925, Validation Accuracy: 0.27041129664910907
Epoch 7, Train Loss: 2.478849843788643, Validation Loss: 2.4691199783774938, Validation Accuracy: 0.26685643841757256
Epoch 8, Train Loss: 2.4757069041829363, Validation Loss: 2.467088827029293, Validation Accuracy: 0.27275579961896224
Epoch 9, Train Loss: 2.477764349897286, Validation Loss: 2.465